### Description:
    This script demonstrates how to load and evaluate a pre-trained sentiment analysis model
    (DistilBERT fine-tuned on SST-2) using PyTorch, Hugging Face’s Transformers, and Datasets libraries.
    It evaluates the model on the SST-2 validation set and prints the overall accuracy along with
    example predictions, without applying any active learning techniques.

### How It Works:
    1. Imports necessary libraries for model loading, dataset handling, and evaluation.
    2. Loads the pre-trained DistilBERT model and its tokenizer using Hugging Face's Auto classes.
    3. Sets up the computation device (GPU if available, otherwise CPU) and puts the model in evaluation mode.
    4. Loads the SST-2 validation dataset from the GLUE benchmark using the Hugging Face Datasets library.
    5. Defines a collate function to tokenize the text and batch the data appropriately.
    6. Creates a DataLoader to handle batch processing of the dataset.
    7. Iterates over the DataLoader, performs a forward pass through the model for each batch,
       collects predictions, and computes the accuracy.
    8. Prints the overall accuracy and a few example predictions with their ground truth labels.

In [1]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import numpy as np

c:\python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
model_name = "distilbert-base-uncased"
# pre-trained model (DistilBERT)

In [3]:
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [5]:
model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [6]:
dataset = load_dataset("glue", "sst2", split="validation")
# Load the SST-2 dataset (using the validation split from GLUE)

In [7]:
def collate_fn(batch):
    texts = [item["sentence"] for item in batch]
    labels = [item["label"] for item in batch]
    tokenized_inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    tokenized_inputs["labels"] = torch.tensor(labels)
    return tokenized_inputs


In [8]:
dataloader = DataLoader(dataset, batch_size=32, collate_fn=collate_fn)

In [9]:
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(dataloader, desc="Evaluating"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
print("Accuracy of the pre-trained model on SST-2 validation set: {:.2f}%".format(accuracy * 100))

Evaluating: 100%|██████████| 28/28 [00:14<00:00,  1.95it/s]

Accuracy of the pre-trained model on SST-2 validation set: 50.92%


In [10]:
for i in range(5):
    sentence = dataset[i]["sentence"]
    true_label = "positive" if dataset[i]["label"] == 1 else "negative"
    pred_label = "positive" if all_preds[i] == 1 else "negative"
    print(f"Sentence: {sentence}")
    print(f"Predicted: {pred_label} | Ground Truth: {true_label}\n")

Sentence: it 's a charming and often affecting journey . 
Predicted: positive | Ground Truth: positive

Sentence: unflinchingly bleak and desperate 
Predicted: positive | Ground Truth: negative

Sentence: allows us to hope that nolan is poised to embark a major career as a commercial yet inventive filmmaker . 
Predicted: positive | Ground Truth: positive

Sentence: the acting , costumes , music , cinematography and sound are all astounding given the production 's austere locales . 
Predicted: positive | Ground Truth: positive

Sentence: it 's slow -- very , very slow . 
Predicted: positive | Ground Truth: negative

